# Alpaca-py stock trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/stocks/stocks-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with stock trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for stocks as an example.

In [1]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

api_key = None
secret_key = None

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [2]:
from dotenv import load_dotenv

load_dotenv()

import os

if api_key is None:
    api_key = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    secret_key = os.environ.get('ALPACA_SECRET_KEY')

In [4]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    %pip install alpaca-py
    import alpaca

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [alpaca-py]
Note: you may need to restart the kernel to use updated packages.


In [5]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.corporate_actions import CorporateActionsClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    CorporateActionsRequest,
    StockBarsRequest,
    StockQuotesRequest,
    StockTradesRequest,
)
from alpaca.trading.requests import (
    ClosePositionRequest,
    GetAssetsRequest,
    GetOrdersRequest,
    LimitOrderRequest,
    MarketOrderRequest,
    StopLimitOrderRequest,
    StopLossRequest,
    StopOrderRequest,
    TakeProfitRequest,
    TrailingStopOrderRequest,
)
from alpaca.trading.enums import (
    AssetExchange,
    AssetStatus,
    OrderClass,
    OrderSide,
    OrderType,
    QueryOrderStatus,
    TimeInForce,
)

In [6]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [7]:
# check version of alpaca-py
alpaca.__version__

'0.42.0'

# Trading Client

In [8]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [9]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3AQ1JOGHJ4',
    'accrued_fees': '0',
    'buying_power': '200000',
    'cash': '100000',
    'created_at': datetime.datetime(2025, 7, 14, 2, 39, 56, 312016, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100000',
    'id': UUID('f36ad74a-f4eb-4f33-b556-939055df4f73'),
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '100000',
    'options_approved_level': 3,
    'options_buying_power': '100000',
    'options_trading_level': 3,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '100000',
    'regt_buying_power': '200000',
    'short_market_value': '0',
    'shorting_enabled': 

In [10]:
# check account configuration
# ref. https://docs.alpaca.markets/reference/getaccountconfig-1
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [11]:
# set account configuration
# ref. https://docs.alpaca.markets/reference/patchaccountconfig-1
req = acct_config
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_new = trade_client.set_account_configurations(req)
display(acct_config_new)

# revert changes
req = acct_config_new
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_reverted = trade_client.set_account_configurations(req)
display(acct_config_reverted)

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': False,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [12]:
# get list of assets which are us_equity (default), active, and in NASDAQ
# ref. https://docs.alpaca.markets/reference/get-v2-assets-1
req = GetAssetsRequest(
  # asset_class=AssetClass.US_EQUITY,  # default asset_class is us_equity
  status=AssetStatus.ACTIVE,
  exchange=AssetExchange.NASDAQ,
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'fractionable': False,
     'id': UUID('47a06f28-06e9-4173-9bc4-e89e049aea10'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'JBDI Holdings Limited Ordinary Shares',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'JBDI',
     'tradable': False},
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'fractionable': False,
     'id': UUID('5a381485-5fa2-4f96-8556-b2a900b51441'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'Top KingWin Ltd Clas

### Orders

In [13]:
# we will place orders which Alapca trading platform supports
# - order classes: simple, bracket, oco, oto
# - order types: market, limit, stop, stop_limit, trailing_stop
#
# please refer to the following documents for more details
# ref. https://docs.alpaca.markets/docs/orders-at-alpaca
# ref. https://docs.alpaca.markets/reference/postorder
#
# we will also use fractional trading capability of Alpaca trading platform in this example
# ref. https://docs.alpaca.markets/docs/fractional-trading

# we will place orders for symbol: SPY in this example
symbol = "SPY"

In [14]:
# simple, market order, fractional qty
# Alpaca trading platform support fractional trading by default
# you can specify:
# fractional qty (e.g. 0.01 qty) in the order request (which is shown in this example)
# or notional value (e.g. 100 USD) (which is in the next example)
#
# If you have an error of `qty must be integer`,
# please try to `Reset Account` of your paper account via the Alpaca Trading API dashboard
req = MarketOrderRequest(
    symbol = symbol,
    qty = 5.5,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '699bedda-4f89-4ad4-9d94-ccb888165956',
    'created_at': datetime.datetime(2025, 7, 14, 3, 57, 23, 818343, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 7, 14, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('110694ac-6baa-46ba-afcf-c383e2dd2bd8'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '5.5',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderSta

In [15]:
# simple, market order, notional
# Alpaca trading platform support fractional trading by default
# you can specify:
# fractional qty (e.g. 0.01 qty) in the order request (which is in the example above)
# or notional value (e.g. 100 USD) (which is in this example)
req = MarketOrderRequest(
    symbol = symbol,
    notional = 1.11,  # notional is specified in USD, here we specify $1.11
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '650dd40f-a5dd-45f4-8f0a-a7f9f7c4eb62',
    'created_at': datetime.datetime(2025, 7, 14, 3, 57, 23, 846059, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 7, 14, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('f3a58299-02d1-42b7-90bf-ea93174b134e'),
    'legs': None,
    'limit_price': None,
    'notional': '1.11',
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': None,
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderSt

In [16]:
# simple, limit order, fractional qty
req = LimitOrderRequest(
    symbol = symbol,
    qty = 0.01,
    limit_price = 550.25,
    side = OrderSide.BUY,
    type = OrderType.LIMIT,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '11b30006-4de5-49c6-9672-57d7b5462345',
    'created_at': datetime.datetime(2025, 7, 14, 3, 57, 23, 874152, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 7, 14, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('aec98d43-1784-46a8-b72e-6dbd155ca86f'),
    'legs': None,
    'limit_price': '550.25',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '0.01',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <Order

In [17]:
# stop order
req = StopOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    stop_price = 600
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '678c65ce-007c-41fc-ab05-aa728948cad2',
    'created_at': datetime.datetime(2025, 7, 14, 3, 57, 23, 897205, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 10, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('6167bfd9-a258-423a-9660-d362737a9275'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP: 'stop'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.A

In [18]:
# stop limit order
req = StopLimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    limit_price = 550,
                    stop_price = 600
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'c650aed7-b2ea-4934-a113-095a84f0df5b',
    'created_at': datetime.datetime(2025, 7, 14, 3, 57, 23, 920035, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 10, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('ec942912-a3ad-4894-9db6-e4c84f63d4e4'),
    'legs': None,
    'limit_price': '550',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <

In [19]:
# bracket order with both stop loss and take profit
req = MarketOrderRequest(
                    symbol = symbol,
                    qty = 5,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    order_class = OrderClass.BRACKET,
                    take_profit = TakeProfitRequest(limit_price=600),
                    stop_loss = StopLossRequest(stop_price=300)
)
res = trade_client.submit_order(req)
res

APIError: {"base_price":"623.62","code":42210000,"message":"take_profit.limit_price must be \u003e= base_price + 0.01"}

In [20]:
# oto order with stop loss
req = LimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    limit_price = 500,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    Class = OrderClass.OTO,
                    stop_loss = StopLossRequest(stop_price = 300)
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '31bbe6e9-85af-466b-8d38-8ac00b876dbe',
    'created_at': datetime.datetime(2025, 7, 14, 3, 58, 18, 773009, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 7, 14, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('169fadec-6bdb-47a9-850c-6020bd50cff4'),
    'legs': None,
    'limit_price': '500',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus

In [21]:
# oco limit order
req = LimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    limit_price = 500,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    Class = OrderClass.OCO
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'bb15b388-0118-409f-bd48-ce3ad0334f5f',
    'created_at': datetime.datetime(2025, 7, 14, 3, 58, 18, 801087, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 7, 14, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('6a3caf54-c9bd-4016-b29c-4e3f9dd319de'),
    'legs': None,
    'limit_price': '500',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus

In [22]:
# trailing stop order
req = TrailingStopOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.SELL,
                    time_in_force = TimeInForce.GTC,
                    trail_percent = 0.20 # you can also use trail_price instead of trail_percent
                    )

res = trade_client.submit_order(req)
res

APIError: {"code":40310000,"message":"cannot open a short sell while a long buy order is open"}

In [ ]:
# get a list of orders including closed (e.g. filled) orders by specifying symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [symbol]
)
orders = trade_client.get_orders(req)
orders

In [ ]:
# see all open orders
req = GetOrdersRequest(
    status = QueryOrderStatus.OPEN,
    symbols = [symbol]
)
open_orders = trade_client.get_orders(req)
open_orders

In [ ]:
# cancel all open orders
trade_client.cancel_orders()

### Positions

In [ ]:
# get all open positions
# ref. https://docs.alpaca.markets/reference/getallopenpositions-1
positions = trade_client.get_all_positions()
positions

In [ ]:
# get positions by symbol
# ref. https://docs.alpaca.markets/reference/getopenposition-1
position = trade_client.get_open_position(symbol_or_asset_id=symbol)
position


In [ ]:
# get positions by asset_id
trade_client.get_open_position(symbol_or_asset_id=position.asset_id)

In [ ]:
# close the position with specifying qty
# ref. https://docs.alpaca.markets/reference/deleteopenposition-1
trade_client.close_position(
    symbol_or_asset_id = symbol,
    close_options = ClosePositionRequest(
        qty = "0.01",
    )
)

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

You can open this notebook in another window and run below cell to check trade updates.

In [ ]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

# Market Data (Historical)

In [23]:
# setup stock historical data client
stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [24]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
stock_historical_data_client.get_stock_bars(req).df

open    high     low   close   volume  \
symbol timestamp                                                            
SPY    2025-07-09 08:00:00+00:00  620.47  620.71  620.35  620.53  21357.0   
       2025-07-09 09:00:00+00:00  620.54  621.60  620.54  621.31  54377.0   

                                  trade_count        vwap  
symbol timestamp                                           
SPY    2025-07-09 08:00:00+00:00        308.0  620.495547  
       2025-07-09 09:00:00+00:00        659.0  621.383605

In [ ]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Minute), # specify timeframe
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    # limit = 2,                                               # specify limit
)
stock_historical_data_client.get_stock_bars(req).df

open    high       low     close   volume  \
symbol timestamp                                                                
SPY    2025-07-09 08:00:00+00:00  620.47  620.47  620.3500  620.3500  11136.0   
       2025-07-09 08:01:00+00:00  620.47  620.47  620.4700  620.4700    271.0   
       2025-07-09 08:02:00+00:00  620.40  620.40  620.4000  620.4000    391.0   
       2025-07-09 08:03:00+00:00  620.41  620.41  620.4100  620.4100    476.0   
       2025-07-09 08:06:00+00:00  620.68  620.68  620.6100  620.6100   1246.0   
...                                  ...     ...       ...       ...      ...   
       2025-07-11 23:55:00+00:00  623.19  623.19  623.1100  623.1100   1671.0   
       2025-07-11 23:56:00+00:00  623.12  623.12  623.1000  623.1000    975.0   
       2025-07-11 23:57:00+00:00  623.05  623.05  622.9501  622.9501   3246.0   
       2025-07-11 23:58:00+00:00  623.12  623.12  623.1200  623.1200    584.0   
       2025-07-11 23:59:00+00:00  623.04  623.04  623.0400  623.0400    554.0   

                                  trade_count        vwap  
symbol timestamp                                           
SPY    2025-07-09 08:00:00+00:00         54.0  620.463133  
       2025-07-09 08:01:00+00:00         13.0  620.470000  
       2025-07-09 08:02:00+00:00         14.0  620.400000  
       2025-07-09 08:03:00+00:00         13.0  620.410000  
       2025-07-09 08:06:00+00:00         19.0  620.628867  
...                                       ...         ...  
       2025-07-11 23:55:00+00:00         11.0  623.115047  
       2025-07-11 23:56:00+00:00         33.0  623.116040  
       2025-07-11 23:57:00+00:00         75.0  623.012602  
       2025-07-11 23:58:00+00:00         20.0  623.120000  
       2025-07-11 23:59:00+00:00         19.0  623.040000  

[2449 rows x 7 columns]

In [35]:
dir(TimeFrameUnit)

['Day',
 'Hour',
 'Minute',
 'Month',
 'Week',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

In [25]:
# get historical trades by symbol
req = StockTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                                # specify limit
)
stock_historical_data_client.get_stock_trades(req).df

exchange   price  size  \
symbol timestamp                                                 
SPY    2025-07-09 08:00:00.008696+00:00        K  620.00   2.0   
       2025-07-09 08:00:00.009755+00:00        K  619.96   5.0   

                                                     id conditions tape  
symbol timestamp                                                         
SPY    2025-07-09 08:00:00.008696+00:00  52983525027889  [ , T, I]    B  
       2025-07-09 08:00:00.009755+00:00  52983525027890  [ , T, I]    B

In [26]:
# get historical quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                      # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                              # specify limit
)
stock_historical_data_client.get_stock_quotes(req).df

bid_price  bid_size bid_exchange  \
symbol timestamp                                                            
SPY    2025-07-09 08:00:00.001031+00:00     620.46       1.0            T   
       2025-07-09 08:00:00.026457+00:00     620.46       1.0            T   

                                         ask_price  ask_size ask_exchange  \
symbol timestamp                                                            
SPY    2025-07-09 08:00:00.001031+00:00     620.55       1.0            T   
       2025-07-09 08:00:00.026457+00:00     620.54       1.0            Z   

                                        conditions tape  
symbol timestamp                                         
SPY    2025-07-09 08:00:00.001031+00:00        [R]    B  
       2025-07-09 08:00:00.026457+00:00        [R]    B

In [27]:
# get latest quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
)
res = stock_historical_data_client.get_stock_latest_quote(req)
res

{'SPY': {   'ask_exchange': 'V',
     'ask_price': 623.58,
     'ask_size': 15.0,
     'bid_exchange': 'V',
     'bid_price': 623.54,
     'bid_size': 15.0,
     'conditions': ['R'],
     'symbol': 'SPY',
     'tape': 'B',
     'timestamp': datetime.datetime(2025, 7, 11, 20, 1, 2, 137327, tzinfo=TzInfo(UTC))}}

# Market Data (Stream)

In [28]:
stock_data_stream_client = StockDataStream(api_key, secret_key, url_override = stream_data_wss)

async def stock_data_stream_handler(data):
    print(data)

symbols = [symbol]

stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

stock_data_stream_client.run()

keyboard interrupt, bye


TimeoutError: 

# Corporate actions

In [29]:
corporate_actions_client = CorporateActionsClient(api_key, secret_key)
corporate_actions_client.get_corporate_actions(CorporateActionsRequest(
    start=datetime(2020, 1, 1),
    symbols=[symbol]
)).df

,id,symbol,cusip,rate,special,foreign,process_date,ex_date,record_date,payable_date
corporate_action_type,,,,,,,,,,
cash_dividends,e06f1a82-04b5-4b90-9c7b-06d22b720db1,SPY,78462F103,1.405559,False,False,2020-04-30,2020-03-20,2020-03-23,2020-04-30
cash_dividends,a6587d80-e08a-4b1a-aa94-857549880c59,SPY,78462F103,1.366242,False,False,2020-07-31,2020-06-19,2020-06-22,2020-07-31
cash_dividends,ffb8940e-53fe-4b3e-a475-e15e39561d3d,SPY,78462F103,1.339224,False,False,2020-10-30,2020-09-18,2020-09-21,2020-10-30
cash_dividends,4d35e709-773b-46d4-b292-34d14c46f287,SPY,78462F103,1.580000,False,False,2021-01-29,2020-12-18,2020-12-21,2021-01-29
cash_dividends,f1ddda1e-b10c-4576-8253-4d8d5a149de6,SPY,78462F103,1.277788,False,False,2021-04-30,2021-03-19,2021-03-22,2021-04-30
cash_dividends,a06ac608-4210-4e4f-94f0-4fa342a8e1fd,SPY,78462F103,1.375875,False,False,2021-07-30,2021-06-18,2021-06-21,2021-07-30
cash_dividends,d2b09118-e4c0-4b93-9f46-10b211400773,SPY,78462F103,1.428117,False,False,2021-10-29,2021-09-17,2021-09-20,2021-10-29
cash_dividends,7cdc9442-1a32-4f40-9d2a-b721edfef9e9,SPY,78462F103,1.636431,False,False,2022-01-31,2021-12-17,2021-12-20,2022-01-31
cash_dividends,f5ca31fd-da0d-4651-be6b-6ac78542259e,SPY,78462F103,1.366009,False,False,2022-04-29,2022-03-18,2022-03-21,2022-04-29
